In [ ]:
pip install tiktoken

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle
import mmap
import random
import os
import logging 
import tiktoken
import re
import unicodedata
import time

In [ ]:
batch_size = 32 # how many independent sequences will we process in parallel?
context_win = 128 # what is the maximum context length for predictions?
learning_rate = 3e-5
device = 'cuda' if torch.cuda.is_available() else 'cpu' #use this when running on nvidia gpu
n_embd = 1024
n_head = 8
n_layer = 12
max_epochs = 5 
dropout = 0.2
vocab_size = 100277 
torch.manual_seed(1337)

In [ ]:
# #only for TPU run
# num_cores = xm.xrt_world_size()
# # Select the TPU device
# device = xm.xla_device()

In [ ]:
enc = tiktoken.get_encoding("cl100k_base")

In [ ]:
eval_interval = 1000
eval_iters = 200

In [ ]:
device

In [ ]:
train_file = r"" #location of training file
val_file = r""

In [ ]:
def get_batch(split):
    global current_position 
    filename = train_file if split == 'train' else val_file
    filesize = os.path.getsize(filename)  # Get size of the file using 'os'
    encodings = ['utf-8']
    for encoding in encodings:
        try:
            with open(filename, 'r', encoding=encoding,errors='replace') as f:
                f.seek(current_position)

                data_chunk = ''
                while len(data_chunk) < context_win * batch_size:
                    additional_data = f.read(context_win * batch_size - len(data_chunk))
                    data_chunk += additional_data

                if not data_chunk:  
                    current_position = 0  
                    return None 
                data = torch.tensor(enc.encode(data_chunk), dtype=torch.long)  
                
                current_position += len(data_chunk)  

                ix = torch.randint(len(data) - context_win, (batch_size,))
                x = torch.stack([data[i:i+context_win] for i in ix])
                y = torch.stack([data[i+1:i+context_win+1] for i in ix])
                x, y = x.to(device), y.to(device)

                return x, y 
        except UnicodeDecodeError:
            print("UnicodeDecodeError")   
    return None

In [ ]:
@torch.no_grad()
def estimate_loss():
        out = {}
        model.eval()
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = get_batch(split)
                if X is None:  # Handle case when get_batch returns None
                    continue
                logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
        model.train()
        return out

In [ ]:
class Head(nn.Module):
    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size)
        self.query = nn.Linear(n_embd,head_size)
        self.value = nn.Linear(n_embd,head_size)
        self.register_buffer('tril',torch.tril(torch.ones(context_win,context_win)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T,:T]==0,float('-inf'))
        wei = F.softmax(wei,dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out
        

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
class GPTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.position_embedding = nn.Embedding(context_win,n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) 
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)    
        
    def forward(self,index,target=None):
        B,T = index.shape
        
        token = self.token_embedding_table(index)
        pos_embd = self.position_embedding(torch.arange(T,device=device))
        x = token + pos_embd
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if target == None:
            loss = None
        else:    
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits,target)
            
        return logits,loss    

            
    def generate(self,index,max_size):
        for _ in range(max_size):
            index_cropped = index[:,-context_win:]
            logits,loss = self(index_cropped)
            logits = logits[:,-1,:]
            prob = F.softmax(logits,dim=-1)
            next_word = torch.multinomial(prob,num_samples=1)
            index = torch.cat((index,next_word),dim = 1)
        return index    

model = GPTModel() 
model = model.to(device)

In [ ]:
#checkpoint = torch.load(r'', map_location=torch.device('cuda')) # load model if already has a model
#model.load_state_dict(checkpoint)

In [ ]:
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
num_batches_per_epoch = 2200000
num_batches_per_epoch

In [ ]:
save_dir = r''
os.makedirs(save_dir, exist_ok=True)

In [ ]:
def save_model(model, counter):
    model_path = os.path.join(save_dir, f'model_iter_{counter}.pt')
    torch.save(model.module.state_dict(), model_path)
    print(f"Model saved at iteration {counter}")

In [ ]:
def calculate_perplexity(model):
    total_loss = 0
    total_tokens = 0
    num_batches_per_validation = 32  # Define the number of batches for validation
    
    # Set the model to evaluation mode
    model.eval()
    
    with torch.no_grad():
        for _ in range(num_batches_per_validation):
            try:
                # Get a batch of validation data using get_batch() function
                xb, yb = get_batch('val')  # Assuming 'val' is the split for validation data
                
                if xb is None:
                    break  # End of validation data
                
                # Forward pass and loss estimation
                losses = estimate_loss()  # Using estimate_loss() function
                loss = losses['val']
                
                # Update total loss and total tokens
                total_loss += loss.item() * yb.numel()  # Multiplying loss by number of tokens
                total_tokens += yb.numel()  # Count the number of tokens in targets (yb)
            
            except Exception as e:
                continue
    
    # Calculate perplexity
    average_loss = total_loss / total_tokens
    perplexity = torch.exp(torch.tensor(average_loss))
    
    return perplexity


In [ ]:
current_position = 0
counter = 0

In [ ]:
for epoch in range(max_epochs):
    for iter in range(num_batches_per_epoch):
        # Sample a batch of data (random or sequential, based on your get_batch implementation)
            start = time.time()
            xb, yb = get_batch('train')
            if xb is None:
                print('none')
                continue
            else:
                # Update the counter for every iteration
                counter += 1

                # Evaluate the loss
                logits, loss = model(xb, yb)
                optimizer.zero_grad(set_to_none=True)
                loss.backward()
                optimizer.step()
                if counter % eval_interval == 0 or epoch == max_epochs - 1:
                                endtime = time.time() - start
                                losses = estimate_loss() 
                                print(f"Iteration {counter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, total time taken: {endtime}")
                                print(enc.decode(model.generate(torch.zeros((1,1),dtype=torch.long,device=device),max_size=500)[0].tolist()))
                                print('\n')
                                print("------------")
                # Check if it's time to save the model
                if counter % 100 == 0 :
                    print(".",end="", flush=True)
                if counter % 5000 == 0:
                    save_model(model, counter)
                    
                # Reset current_position if we've reached the end of the epoch 
                if iter == num_batches_per_epoch - 1: 
                    current_position = 0
 

In [ ]:
current_position,counter

In [ ]:
perplexity = calculate_perplexity(model)
print(f"Iteration {counter}: Perplexity: {perplexity}")

In [ ]:
print(enc.decode(model.generate(torch.zeros((1,1),dtype=torch.long,device=device),max_size=1500)[0].tolist()))

In [ ]:
print(f"Iteration {counter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

In [ ]:
save_model(model, counter)